In [ ]:
import logging

from src.common import DataSource
from src.container import ApplicationContainer

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

di_container = ApplicationContainer()  # DI Container 초기화 및 자동 와이어링

upbit_api = di_container.upbit_api()
candle_service = di_container.candle_service()

hantu_domestic_api = di_container.hantu_domestic_api()
# hantu_overseas_api = HantuOverseasAPI(HantuConfig())  # type: ignore

# v_hantu_domestic_api = HantuDomesticAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore
# v_hantu_overseas_api = HantuOverseasAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore

# auto_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "auto_data")
# kimp_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "kimp_data")

# bithumb_config = BithumbConfig()

In [2]:
%env ENV_PROFILE=local
from src.container import ApplicationContainer
from src.upbit.upbit_api import UpbitCandleInterval
from src.constants import KRW_BTC
from src.common import DataSource

di_container = ApplicationContainer()
upbit_api = di_container.upbit_api()
candle_service = di_container.candle_service()

result = upbit_api.get_candles(market=KRW_BTC, interval=UpbitCandleInterval.MINUTE_1, count=1)
# result
candle_service.save_candles(result, DataSource.UPBIT, KRW_BTC, UpbitCandleInterval.MINUTE_1)

# candle_service.collect_minute1_candles(ticker=KRW_BTC)

env: ENV_PROFILE=local


In [4]:
%env ENV_PROFILE=local
from src.database import CandleMinute1Repository

minute__repository: CandleMinute1Repository = di_container.candle_minute1_repository()
minute__repository.get_candles(KRW_BTC)

env: ENV_PROFILE=local


[<CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:31:00+09:00, close=134332000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:32:00+09:00, close=134332000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:33:00+09:00, close=134332000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:34:00+09:00, close=134332000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:35:00+09:00, close=134371000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:36:00+09:00, close=134363000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:37:00+09:00, close=134297000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:38:00+09:00, close=134361000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:39:00+09:00, close=134280000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:40:00+09:00, close=134340000.0)>,
 <CandleMinute1(ticker=KRW-BTC, timestamp=2025-12-06 14:41:00+09:00, close=134335000.0)>,
 <CandleMi